# Tensorboard with MNIST MLP
#### Debugging a neural network with tensorboard
Try playing with the neural network hyperparameters such as learning rate, batch size, activation functions and see what happens to the Tensorboard output. 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### Fetching MNIST 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

### Sizes of Input, Output and Hidden Layers 

In [ ]:
n_inputs = 784
n_outputs = 10
n_hidden_1 = 256
n_hidden_2 = 256

### Input and Output Placeholders 

In [ ]:
batch_x = tf.placeholder(dtype=tf.float32, shape=[None, n_inputs], name="Batch_X")
batch_y = tf.placeholder(dtype=tf.float32, shape=[None, n_outputs], name="Batch_Y")

### Tensboard summary ops
Tensorboard provides summary operation that you can apply onto tensors to visualize them over training steps. 
Few of them are listed below:
1. tf.summary.scalar(): For scalars such as loss, accuracy. 
2. tf.summary.image(): For image matrices such as a 28x28 MNIST input image.
3. tf.summary.histogram(): For numeric distributions used for weights, biases, activations.

### Weights and Biases  

In [ ]:
Wh1i = tf.Variable(tf.random_normal((n_inputs, n_hidden_1)), dtype=tf.float32, name="Wh1")
tf.summary.histogram('Wh1', Wh1i)
bh1 = tf.Variable(tf.random_normal((1, n_hidden_1)), dtype=tf.float32, name="Bh1")
tf.summary.histogram('Bh1', bh1)

Wh2h1 = tf.Variable(tf.random_normal((n_hidden_1, n_hidden_2)), dtype=tf.float32, name="Wh2")
tf.summary.histogram('Wh2h1', Wh2h1)
bh2 = tf.Variable(tf.random_normal((1, n_hidden_2)), dtype=tf.float32, name="Bh2")
tf.summary.histogram('Bh2', bh2)

Woh2 = tf.Variable(tf.random_normal((n_hidden_2, n_outputs)), dtype=tf.float32, name="Wo")
tf.summary.histogram('Woh2', Woh2)
bo = tf.Variable(tf.random_normal((1, n_outputs)), name="Bo")
tf.summary.histogram('Bo', bo)

### Name scopes
Tensorboard can generate a visual computation graph output for the graph that you construct with tensorflow. But the graph that is created by default won't make much sense. You can try it by adding just these two lines to the end of the session.
1. writer = tf.summary.FileWriter("/path/to/save/")
2. writer.add_graph(sess.graph)

In order to better visualize the graph you can scope related operations into semantic layers which you will see below. To do this you need to use tf.name_scope(). 

You can also supply name parameter to the desired tensor ops, variables, etc.

### Tensor Operations
We are using ReLU as activation function and softmax at the final layer to get max probability.

In [ ]:
with tf.name_scope("Layer_1"):
    zh1 = tf.add(tf.matmul(batch_x, Wh1i), bh1)
    # ah1 = tf.nn.tanh(zh1)
    # ah1 = tf.nn.relu(zh1)
    # ah1 = tf.nn.sigmoid(zh1)
    ah1 = zh1

with tf.name_scope("Layer_2"):
    zh2 = tf.add(tf.matmul(ah1, Wh2h1), bh2)
    # ah2 = tf.nn.tanh(zh2)
    # ah2 = tf.nn.relu(zh2)
    # ah2 = tf.nn.sigmoid(zh2)
    ah2 = zh2

with tf.name_scope("Layer_Output"):
    zo = tf.add(tf.matmul(ah2, Woh2), bo)
    # ao = tf.nn.tanh(zo)
    # ao = tf.nn.relu(zo)
    # ao = tf.nn.sigmoid(zo)
    ao = zo

In [ ]:
logits = ao
prediction = tf.nn.softmax(ao)

In [ ]:
with tf.name_scope("Loss"):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=batch_y))
    tf.summary.scalar('Loss', loss_op)
with tf.name_scope("Train"):
    train_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss_op)

### Number of training_steps 

In [ ]:
training_epochs = 15
display_step = 10
batch_size = 100

### Initialize variables 

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
writer = tf.summary.FileWriter("/tmp/mnist_demo/3") # change the path everytime you rerun the session to properly see the outputs 

In [ ]:
with tf.Session() as sess:
    # tensorboard settings
    merged_summary = tf.summary.merge_all()
    writer.add_graph(sess.graph)
    
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            x, y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            s, _, c = sess.run([merged_summary, train_op, loss_op], feed_dict={batch_x: x, 
                                                         batch_y: y})
            writer.add_summary(s, i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
#     pred = tf.nn.softmax(logits)  # Apply softmax to logits
    with tf.name_scope("Test_accuracy"):
        correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(batch_y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tf.summary.scalar('Accuracy', accuracy)
    print("Accuracy:", accuracy.eval({batch_x: mnist.test.images, batch_y: mnist.test.labels}))
    
    # save the trained model
    save_path = saver.save(sess, "/tmp/model-mnist.ckpt")

### Checking Tensorboard Output
To run tensorboard you can do:
python -m tensorboard.main --logdir="/path/to/save/file"

### Run predictions 

In [ ]:
import random

In [ ]:
n_images = 10
start = random.randint(0, mnist.test.num_examples - n_images)
test_images = mnist.test.images[start:start + n_images - 1]
test_labels = mnist.test.labels[start:start + n_images - 1]
for idx, image in enumerate(test_images):
    image = np.reshape(image, (28, 28))
    print(np.argmax(test_labels[idx]), end=',')
    plt.subplot(1,n_images,idx+1)
    plt.imshow(image, cmap='gray')
    plt.draw()

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model-mnist.ckpt")
    _logits = sess.run(logits, feed_dict={batch_x: test_images})
    _pred = tf.nn.softmax(_logits)
    predicted_number = tf.argmax(_pred, 1)
    _predicted_number = predicted_number.eval(feed_dict={batch_x: test_images})
    _actual_number = np.argmax(test_labels, 1)
    print('Predicted:',_predicted_number,'Actual:',_actual_number)